In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
from openai import OpenAI

openai = OpenAI() # a new instance of the OpenAI Python Client library, a lightweight wrapper around making HTTP calls to an endpoint for calling the GPT LLM, or other LLM providers

#### A Message to OpenAI:

In [4]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Betül!"}
    ]

In [5]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'Hello Betül! Nice to meet you. How can I assist you today?'

#### Follow-up Question:

In [6]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What's my name?"}
    ]

In [7]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'I’m not sure what your name is. Could you please tell me?'

#### LLMs Themselves Have No Permanent Memory

A base language model (like GPT-4, Claude, Gemini, etc.) is stateless.
That means:
* It does not “remember” anything from past conversations once the session ends.
* It processes only the current input tokens (your message + recent history).
* When the chat is closed, that data is gone.


So the model’s parameters (the “weights”) are fixed — they don’t change while chatting.
It’s not like a human brain that stores new experiences.

#### Then How Does It “Remember” During a Conversation?

During an ongoing chat (one session), everything me and Chat say is part of a context window.
It is like a temporary short-term memory.
The model doesn’t “store” your name permanently — it just reads it again from the chat history still in the _context window_.
That window might hold tens of thousands of tokens (for GPT-4-turbo, up to ~128k tokens).
Once the context gets too long or the chat resets, that information disappears.

When you use ChatGPT (the app), OpenAI adds a “memory” feature:

* Certain facts you share (like your department, projects, or interests) can be stored by the system.

* When you return in a new chat, that info is automatically loaded into Chat's context.

* The model itself didn’t remember — the platform did.

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Betül!"},
    {"role": "assistant", "content": "Hi Betül! How can I assist you today?"},
    {"role": "user", "content": "What's my name?"}
    ]

In [9]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'Your name is Betül! How can I help you today, Betül?'

1. Every call to an LLM is stateless
2. We pass in the entire conversation so far in the input prompt, every time
3. This gives the illusion that the LLM has memory - it apparently keeps the context of the conversation
4. But this is a trick; it's a by-product of providing the entire conversation, every time
5. An LLM just predicts the most likely next tokens in the sequence; if that sequence contains "My name is Betül" and later "What's my name?" then it will predict.. Betül!

The ChatGPT product uses exactly this trick - every time you send a message, it's the entire conversation that gets passed in.